In [1]:
import os
from haystack import Pipeline, component
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret
from haystack.components.generators.chat import OpenAIChatGenerator
from typing import List, Dict

from sklearn.metrics import accuracy_score

In [2]:
@component
class QueryClassifierLLM:
    """
    Классифицирует, нужен ли поиск (RAG) для запроса,
    посредством LLM, возвращающей булев результат.
    """
    def __init__(self, generator: OpenAIChatGenerator):
        self.generator = generator
        self.template = """
Ты – классификатор запросов по базе знаний. 
Если запрос требует поиска информации – ответь 'true'. 
Если можно ответить без поиска – ответь 'false'.

Примеры:
#1
Запрос: "Привет"
Ответ: false

#2
Запрос: "Как дела?"
Ответ: false

#3
Запрос: "Спасибо за помощь"
Ответ: false

#4
Запрос: "Что такое GDPR?"
Ответ: true

#5
Запрос: "Сколько дней отпуска положено сотрудникам?"
Ответ: true

#6
Запрос: "Расскажи, пожалуйста, о новых политиках отпуска"
Ответ: true

Теперь классифицируй новый запрос:
Запрос: "{{ query }}"
Ответ:
""".strip()

    @component.output_types(need_search=bool)
    def run(self, query: str) -> dict:
        # 1️⃣ Подставляем запрос в шаблон
        prompt = self.template.replace("{{ query }}", query)

        # 2️⃣ Упаковываем в ChatMessage
        msg = ChatMessage.from_user(prompt)

        # 3️⃣ Вызываем генератор; он отдаёт словарь с ключом "replies"
        output = self.generator.run([msg])
        replies = output.get("replies", [])

        # 4️⃣ Извлекаем текст первого ответа (или пустую строку)
        text = replies[0].text.strip().lower() if replies else ""

        # 5️⃣ Булево решение на основе префикса "true"
        need_search = text.startswith("true")
        return {"need_search": need_search}

In [3]:
# 2. Настраиваем LLM‑генератор (Ollama/OpenAI compatible)
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0"
llm = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),            # предполагается, что токен настроен
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={"temperature": 0.0}
)

In [4]:
# 3. Собираем тестовый Pipeline
test_pipe = Pipeline()
test_pipe.add_component("classifier", QueryClassifierLLM(generator=llm))

In [5]:
# 3) Набор тестовых запросов
test_queries = [
    ("Привет", False),
    ("Что такое GDPR?", True),
    ("Как дела?", False),
    ("Сколько дней отпуска?", True),
    ("Привет", False),
    ("Как дела?", False),
    ("Расскажи, пожалуйста, о новых политиках отпуска", True),
    ("Спасибо", False),
    ("Сколько дней отпуска положено сотрудникам?", True),
    ("Пока", False),
    ("Можно ли мне получить отпуск в июле?", True),
    ("Что ты умеешь?", False),
    ("Где найти инструкцию по подаче заявления?", True),
    ("Покажи мне настройки профиля.", True),
    ("Как мне изменить пароль?", True),
        # Новые исторические/искусствоведческие вопросы
    ("Как менялось предназначение здания музея Прадо на протяжении истории, и какие ключевые события повлияли на его становение?", True),
    ("Чем творчество Ханса Бальдунга Грина отличалось от искусства итальянского Ренессанса, и как в его работах проявлялись средневековые традиции?", True),
    ("Как придворная служба повлияла на творчество Лукаса Кранаха Старшего, и какие особенности его работы \"Охота на оленей и кабанов\" отражают его положение при саксонском дворе?", True),
    ("Как в портрете шута \"Эль Примо\" Диего Веласкес сочетает психологическую глубину с социальным контекстом, и какие художественные приёмы помогают раскрыть личность этого придворного?", True),
    ("Как в картине \"Даная\" Тициан переосмысливает античный миф и какие новаторские живописные приёмы позволяют считать эту работу поворотной в его творчестве?", True),
    ("Как в картине \"Парнас\" Никола Пуссен сочетает традиции итальянского Возрождения с собственным художественным видением, и какие элементы раскрывают его диалог с великими предшественниками?", True),
    ("Как скульптура куроса отражает ключевые принципы архаического периода древнегреческого искусства и чем она отличается от классической греческой скульптуры?", True),
    ("Как Клод Лоррен в картине «Прибытие Клеопатры в Тарс» трансформирует историческое событие в пейзажную живопись, и какие художественные приёмы помогают ему создать эффект «волшебного света»?", True),
    ("Как в картине «Свобода на баррикадах» Делакруа сочетает аллегорический образ Свободы с реалистичным изображением революционных событий, и какие художественные приёмы подчеркивают романтический характер произведения?", True),
    ("Как в статуе фараона Хафры воплощены основные принципы древнеегипетского царского искусства, и какие художественные приёмы подчёркивают божественный статус правителя?", True),
    ("Как рельеф с изображением музыкантов из гробницы Ненхеферка отражает роль музыки в древнеегипетской погребальной культуре и какие уникальные детали исполнительской практики он сохранил?", True),
    ("Как золотая голова сокола-Хора из Иераконполя отражает религиозно-политическую концепцию древнеегипетской власти и какие уникальные художественные техники использовались при её создании?", True),
    ("Как деревянная модель \"Сцена учета скота\" из гробницы Мекетры отражает социальную иерархию и хозяйственную жизнь Древнего Египта эпохи Среднего царства, и в чем заключалась её сакральная функция?", True),
    ("Какие религиозные и художественные особенности погребальной маски Туйи свидетельствуют о её высоком статусе и отражают древнеегипетские представления о загробной жизни?", True),
    ("Как нагрудное украшение Псусеннеса I сочетает в себе религиозную символику, политическую идеологию и ювелирное мастерство XXI династии?", True),
    ("Как в работе Боттичелли «Благовещение» сочетаются черты позднего кватроченто и уникальный авторский стиль художника, отражающий духовные искания эпохи?", True),
    ("Как в работе Мантеньи «Поклонение пастухов» сочетаются новаторские приёмы Ренессанса и уникальный авторский стиль, передающий новое осмысление библейского сюжета?", True),
    ("Как в \"Алтаре Мероде\" Робера Кампена сочетаются средневековая символика и ренессансные новации, создавая уникальный сплав художественных традиций?", True),
    ("Какие особенности картины Питера Брейгеля Старшего «Жатва» отражают его уникальный стиль и влияние эпохи Возрождения?", True),
    ("Как Ван Гог использовал цвет в портрете «Арлезианка», чтобы выразить свои художественные идеи и эмоциональное состояние?", True),
    ("Как Ян Вермер Делфтский в картине «Молодая женщина с кувшином у окна» передает атмосферу уюта и гармонии домашней жизни?", True),
    ("Как Рембрандту в картине «Аристотель перед бюстом Гомера» удаётся соединить историческую тему с глубоким психологизмом и вневременным философским смыслом?", True),
    ("Как Питер де Хох в картине «Визит» передаёт интимность момента и повседневную поэзию бюргерской жизни через композицию и колорит?", True),
    ("Как Антонис ван Дейк сочетает парадную торжественность и человеческую теплоту в «Портрете Джеймса Стюарта», создавая живой образ аристократа?", True),
    ("Как Эль Греко в картине «Вид Толедо» трансформирует реальный город в драматическое мистическое пространство, отражающее испанский религиозный дух?", True),
    ("Как Веласкес в «Портрете Хуана де Пареха» сочетает строгую монохромную палитру с глубокой психологической характеристикой модели, раскрывая их творческую связь?", True),
    ("Как в «Портрете Гертруды Стайн» Пикассо предвосхищает кубизм, отходя от реалистичного изображения к концептуальной трактовке образа?", True),
    ("Как Джон Констебл в картине «Собор в Солсбери» воплощает свой принцип «смиренного ученичества перед природой», сочетая естественность пейзажа с гармонией архитектуры?", True),
    ("Как Уильям Тернер в акварели «Озеро Цуг» сочетает романтическое восприятие природы с новаторской техникой, предвосхищающей импрессионизм?", True),
]
negative_tests = [
    # 1. Простые реплики
    "Привет, рада тебя слышать!", 
    "До скорого!", 
    "Пока, спасибо за помощь.",
    
    # 2. Благодарности и извинения
    "Спасибо большое!", 
    "Извини, я опаздываю.",
    
    # 3. Общие фразы без информационного запроса
    "Как ваше настроение сегодня?", 
    "Хорошего тебе вечера!",
    
    # 4. Риторические вопросы или безопасные фразы
    "Ты умеешь шутить?", 
    "Ты можешь рассказать анекдот?",
    
    # 5. Уточняющие контекстные фразы (без обращения к БЗ)
    "Повторите, пожалуйста, последнее.", 
    "Что ты имеешь в виду?",
    
    # 7. Задачи интерфейса, а не знаний
    "Открой меню.", 
    "Листай дальше.",
]
test_cases = [
    # ... ваши предыдущие позитивные и негативные примеры ...
    *[(q, False) for q in negative_tests],
]

test_queries.extend(test_cases)
# 2) Собираем списки истинных (y_true) и предсказанных (y_pred) меток
y_true = [expect for _, expect in test_queries]
y_pred = []
for query, _ in test_queries:
    out = test_pipe.run({"query": query})
    pred = out["classifier"]["need_search"]
    y_pred.append(pred)

# 3) Вычисляем accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2%}\n")

# 4) Находим и печатаем несовпадения
mismatches = []
for (query, expect), pred in zip(test_queries, y_pred):
    if pred != expect:
        mismatches.append((query, expect, pred))

if mismatches:
    print("Мismatched cases:")
    for query, expect, pred in mismatches:
        print(f"  • Запрос: {query!r}")
        print(f"    Ожидается need_search={expect}, получено {pred}\n")
else:
    print("Все предсказания совпадают с разметкой.")

Accuracy: 100.00%

Все предсказания совпадают с разметкой.
